In [1]:
import numpy as np
from numba import njit
import timeit
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint

p = print

#pls take note of version
#numba 0.49.1 #https://github.com/numba/numba/issues/4255
#numpy 1.17.3
#pandas 1.0.3
#sklearn 0.21.3

dollar = pd.read_csv('./Sample_data/dollar_bars.txt', 
                 sep=',', 
                 header=0, 
                 parse_dates = True, 
                 index_col=['date_time'])

# For most part of the func we only use 'close'

close = dollar['close'].to_frame()

C:\Users\Wei_X\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
@njit
def getWeights(d, size):
    w=[1.]
    for k in np.arange(1,size):
        w_ = -w[-1]/k*(d-k+1)
        w.append(w_)
    w=np.array(w[::-1]).reshape(-1,1)
    return w

def fracDiff(series, d, thres=.01):
    w=getWeights(d, series.shape[0])
    w_=np.cumsum(abs(w))
    w_/=w_[-1]
    skip = w_[w_>thres].shape[0]
    df={}
    for name in series.columns:
        seriesF,df_=series[[name]].fillna(method='ffill').dropna(),pd.Series(index=series.index, dtype=float)
        for iloc in range(skip,seriesF.shape[0]):
            loc=seriesF.index[iloc]
            if not np.isfinite(series.loc[loc,name]): continue
            df_[loc]=np.dot(w[-(iloc+1):,:].T,seriesF.loc[:loc])[0,0]
        df[name]=df_.copy(deep=True)
    df=pd.concat(df,axis=1)
    return df

@njit
def getWeights_FFD(d, thres):
    w,k=[1.],1
    while True:
        w_ = -w[-1]/k*(d-k+1)
        if abs(w_) < thres:
            break
        w.append(w_); k+=1
    return np.array(w[::-1]).reshape(-1,1)

# need to refactor to optimze if not will take forever if threshold too low
def fracDiff_FFD(series, d, thres=1e-2):
    w, df = getWeights_FFD(d, thres), {}
    width = len(w)-1
    for name in series.columns:
        seriesF,df_=series[[name]].fillna(method='ffill').dropna(),pd.Series(index=series.index, dtype=float)
        for iloc in range(width, seriesF.shape[0]):
            loc0, loc1 = seriesF.index[iloc - width], seriesF.index[iloc]
            if not np.isfinite(series.loc[loc1,name]): continue
            df_[loc1]=np.dot(w.T,seriesF.loc[loc0:loc1])[0,0]
        df[name]=df_.copy(deep=True)
    df=pd.concat(df,axis=1)
    return df

def min_value(data: pd.Series, func, thres = 0.01, pval_threshold: float = 0.05):
    d_domain = np.linspace(start = 0, 
                           stop = 2, 
                           num=100, 
                           endpoint=True, 
                           retstep=False, 
                           dtype=float)
    
    for d in d_domain:
        df1 = np.log(data).resample('1D').last() # pls note downcast to daily obs
        df2 = func(df1, d, thres = thres).dropna()
        df2 = adfuller(df2.squeeze(), maxlag=1, regression='c', autolag=None)
        try:
            if df2[1] <= pval_threshold:
                return d
        except:
            p('Something is wrong! Most likely required d value more than 2!!')

In [3]:
ffd_series = close.apply(np.log).cumsum()
ffd_series = fracDiff_FFD(ffd_series, 
                    d = 1.99999889, 
                    thres=1e-5
                   ).dropna()

adf_pval = adfuller(ffd_series.squeeze(), 
                    maxlag = 1,
                    regression='c', 
                    autolag=None)[1]

p("\nADF pVal: {0:.5f} with critical value: {1}%".format(adf_pval, 5)) #confirm stationarity


ADF pVal: 0.04165 with critical value: 5%


In [4]:
from mlfinlab.filters import filters
from mlfinlab.labeling import labeling



Support us on Patreon: https://www.patreon.com/HudsonThames

MlFinLab needs you! We need your help for us to keep on maintaining and implementing academic research based on 
financial machine learning (for open-source). In order for us to continue we need to raise $4000 of monthly donations
via Patreon - by December 2020. If we can't reach our goal, we will need to adopt more of a paid for service. We thought
that the best and least impactful course of action (should we not reach our goal) is to leave the package as open-source
but to make the documentation (ReadTheDocs) a paid for service. This is the ultimate litmus test, if the package is a 
value add, then we need the community to help us keep it going.

Our road map for 2020 is to implement the text book: Machine Learning for Asset Managers by Marcos Lopez de Prado, 
as well as a few papers from the Journal of Financial Data Science. We are hiring a full time developer for 3 months 
to help us reach our goals. The money that you

In [ ]:
vol = ffd_series.std()
ffd_series.head()


def cumsum_events(data: pd.Series, limit: float):
    '''
    params: pd.Series => time series input
    params: pd.Series => threshold before logging datetime index
    
    This is not the original cumsum event in AFML pg 39 snippet 2.4
    Most part of the code has been rewritten in numpy instead
    
    Logic is the same, but kindly go through the code before using it.
    
    Currently using the below:
    
    #numba 0.49.1
    #numpy 1.17.3
    #pandas 1.0.3
    '''
    if data.isnull().values.any():
        raise ValueError('Data contain NaNs, kindly review data input!')
    idx, _up, _dn, count = [], 0, 0, 0
    diff = data.diff().dropna().to_numpy()
    datetimeidx = data.index[1:].to_numpy()
    df0 = list(zip(datetimeidx, diff))
    for i in np.arange(len(diff)):
        _up, _dn = max(0, _up + df0[i][1]), min(0, _dn + df0[i][1])
        p("up", _up, "_dn", _dn)
        if np.greater(_up, limit).bool() == True:
            _up = 0
            idx.append((df0)[i][0])
        elif np.less(_dn, -limit).bool() == True:
            _dn = 0 
            idx.append((df0)[i][0])
    return pd.DatetimeIndex(idx)


cusum_event = cumsum_events(ffd_series, limit=vol*2)

check = ffd_series.reindex(cusum_event)
#ffd_series[ffd_series['close'] > vol]

up [0.00193557] _dn 0
up [0.00302212] _dn 0
up [0.00204782] _dn [-0.0009743]
up [0.00240654] _dn [-0.00061558]
up [0.0004652] _dn [-0.00255692]
up 0 _dn [-0.00365492]
up [0.00230033] _dn [-0.0013546]
up [0.00302443] _dn [-0.0006305]
up [0.00022687] _dn [-0.00342805]
up [0.00180112] _dn [-0.0018538]
up [0.00070063] _dn [-0.0029543]
up [0.0003277] _dn [-0.00332723]
up [0.00251906] _dn [-0.00113586]
up [0.00336858] _dn [-0.00028634]
up 0 _dn [-0.00382987]
up [0.00096946] _dn [-0.00286041]
up [0.00280007] _dn [-0.00102981]
up [0.00475074] _dn 0
up [0.00083631] _dn [-0.00391443]
up [0.00327573] _dn [-0.00147501]
up [0.00436867] _dn [-0.00038207]
up [0.00106975] _dn [-0.00368099]
up [0.00130819] _dn [-0.00344255]
up [0.0020366] _dn [-0.00271414]
up [0.00080839] _dn [-0.00394235]
up [0.00288458] _dn [-0.00186616]
up [0.0043471] _dn [-0.00040364]
up [0.00250855] _dn [-0.00224219]
up [0.00262643] _dn [-0.00212431]
up [0.00225582] _dn [-0.00249492]
up [0.00212942] _dn [-0.00262132]
up [0.0043221

up [0.00440395] _dn [-0.0072513]
up [0.00232544] _dn [-0.00932981]
up [0.00207613] _dn [-0.00957913]
up [0.00207068] _dn [-0.00958457]
up [0.00402267] _dn [-0.00763259]
up [0.00242895] _dn [-0.00922631]
up [0.00230177] _dn [-0.00935348]
up [0.00364361] _dn [-0.00801165]
up [0.00119026] _dn [-0.010465]
up [0.00339077] _dn [-0.00826449]
up [0.00277362] _dn [-0.00888163]
up [0.00141915] _dn [-0.01023611]
up [0.00227281] _dn [-0.00938244]
up [0.00226784] _dn [-0.00938741]
up [0.00435402] _dn [-0.00730123]
up [0.00336597] _dn [-0.00828929]
up 0 _dn [-0.01223156]
up [0.00171326] _dn [-0.0105183]
up [0.0023239] _dn [-0.00990766]
up [0.00405071] _dn [-0.00818085]
up [0.0023141] _dn [-0.00991746]
up [0.00429018] _dn [-0.00794138]
up [0.00044394] _dn [-0.01178762]
up [0.00142531] _dn [-0.01080625]
up [0.00403175] _dn [-0.00819981]
up [0.00402739] _dn [-0.00820417]
up [0.00389872] _dn [-0.00833284]
up [0.00115642] _dn [-0.01107514]
up [0.00376623] _dn [-0.00846533]
up [0.00139188] _dn [-0.0108396

up [0.01335684] _dn [-0.00898424]
up [0.00905532] _dn [-0.01328576]
up [0.01175176] _dn [-0.01058933]
up [0.01014468] _dn [-0.0121964]
up [0.01334501] _dn [-0.00899608]
up [0.00877387] _dn [-0.01356721]
up [0.01309195] _dn [-0.00924913]
up [0.01308718] _dn [-0.00925391]
up [0.01406921] _dn [-0.00827188]
up [0.01246073] _dn [-0.00988036]
up [0.01479567] _dn [-0.00754542]
up [0.01404911] _dn [-0.00829198]
up [0.01158794] _dn [-0.01075315]
up [0.01342558] _dn [-0.00891551]
up [0.01366526] _dn [-0.00867583]
up [0.01292464] _dn [-0.00941644]
up [0.01414328] _dn [-0.00819781]
up [0.01108039] _dn [-0.0112607]
up [0.01070694] _dn [-0.01163414]
up [0.0133979] _dn [-0.00894318]
up [0.01216847] _dn [-0.01017262]
up [0.01106122] _dn [-0.01127987]
up [0.0120372] _dn [-0.01030389]
up [0.01264614] _dn [-0.00969495]
up [0.01264178] _dn [-0.00969931]
up [0.01202463] _dn [-0.01031646]
up [0.01202034] _dn [-0.01032075]
up [0.0139766] _dn [-0.00836449]
up [0.01311363] _dn [-0.00922745]
up [0.01323087] _dn

up [0.01265262] _dn [-0.00968846]
up [0.00789576] _dn [-0.01444532]
up [0.01089378] _dn [-0.01144731]
up [0.01276771] _dn [-0.00957337]
up [0.01163592] _dn [-0.01070516]
up [0.00963032] _dn [-0.01271077]
up [0.01187814] _dn [-0.01046294]
up [0.01362119] _dn [-0.00871989]
up [0.01099378] _dn [-0.0113473]
up [0.01273455] _dn [-0.00960654]
up [0.01061168] _dn [-0.01172941]
up [0.00960981] _dn [-0.01273128]
up [0.01060258] _dn [-0.01173851]
up [0.009349] _dn [-0.01299209]
up [0.01296646] _dn [-0.00937462]
up [0.01233556] _dn [-0.01000553]
up [0.01208061] _dn [-0.01026047]
up [0.01145344] _dn [-0.01088764]
up [0.01144898] _dn [-0.01089211]
up [0.0119414] _dn [-0.01039969]
up [0.0128043] _dn [-0.00953678]
up [0.00870512] _dn [-0.01363597]
up [0.00944272] _dn [-0.01289837]
up [0.00793982] _dn [-0.01440127]
up [0.01105104] _dn [-0.01129005]
up [0.00829818] _dn [-0.01404291]
up [0.01254291] _dn [-0.00979817]
up [0.00703398] _dn [-0.0153071]
up [0.01228773] _dn [-0.01005336]
up [0.01128074] _dn 

up [0.00911586] _dn [-0.01322523]
up [0.0092333] _dn [-0.01310778]
up [0.00874101] _dn [-0.01360007]
up [0.00995645] _dn [-0.01238463]
up [0.0093423] _dn [-0.01299879]
up [0.00884978] _dn [-0.0134913]
up [0.01043202] _dn [-0.01190906]
up [0.00713096] _dn [-0.01521012]
up [0.0110356] _dn [-0.01130548]
up [0.0095651] _dn [-0.01277599]
up [0.01078112] _dn [-0.01155996]
up [0.00955664] _dn [-0.01278445]
up [0.01077117] _dn [-0.01156991]
up [0.00930454] _dn [-0.01303655]
up [0.0103964] _dn [-0.01194468]
up [0.01051306] _dn [-0.01182802]
up [0.00904892] _dn [-0.01329217]
up [0.0101394] _dn [-0.01220168]
up [0.00928379] _dn [-0.0130573]
up [0.00915783] _dn [-0.01318325]
up [0.00988348] _dn [-0.0124576]
up [0.01085129] _dn [-0.0114898]
up [0.01035995] _dn [-0.01198114]
up [0.01047624] _dn [-0.01186484]
up [0.00998646] _dn [-0.01235463]
up [0.01034529] _dn [-0.01199579]
up [0.00973527] _dn [-0.01260582]
up [0.00912568] _dn [-0.0132154]
up [0.00863664] _dn [-0.01370444]
up [0.00972279] _dn [-0.0

...

up [0.00192737] _dn [-0.04946979]
up [0.00192277] _dn [-0.04947439]
up [0.0027356] _dn [-0.04866156]
up [0.00249786] _dn [-0.0488993]
up [0.00284372] _dn [-0.04855344]
up [0.00248935] _dn [-0.04890781]
up [3.16766673e-05] _dn [-0.05136548]
up [0.00166169] _dn [-0.04973547]
up [0.00306175] _dn [-0.04833541]
up [0.00223829] _dn [-0.04915887]
up [0.00153141] _dn [-0.04986575]
up [0.00304933] _dn [-0.04834783]
up [0.00081925] _dn [-0.05057791]
up [0.00092975] _dn [-0.05046741]
up [0.00245104] _dn [-0.04894612]
up [0.00326853] _dn [-0.04812863]
up [0.00349807] _dn [-0.04789909]
up [0.00314136] _dn [-0.0482558]
up [0.00301955] _dn [-0.04837761]
up [0.00301495] _dn [-0.04838221]
up [0.00301035] _dn [-0.04838681]
up [0.00242127] _dn [-0.04897589]
up [0.00112994] _dn [-0.05026722]
up [0.00194432] _dn [-0.04945284]
up [0.00158821] _dn [-0.04980895]
up [0.0019352] _dn [-0.04946196]
up [0.00322067] _dn [-0.04817649]
up [0.00216133] _dn [-0.04923583]
up [0.00203976] _dn [-0.0493574]
up [0.00168342]

up [0.00217552] _dn [-0.05068427]
up [0.00263621] _dn [-0.05022357]
up [0.00298069] _dn [-0.04987909]
up [0.0033248] _dn [-0.04953498]
up [0.00262344] _dn [-0.05023635]
up [0.00319973] _dn [-0.04966005]
up [0.00215051] _dn [-0.05070928]
up [0.00075084] _dn [-0.05210894]
up [0.00597477] _dn [-0.04688501]
up [0.00318175] _dn [-0.04967803]
up 0 _dn [-0.05316331]
up [0.00371039] _dn [-0.04945292]
up [0.0033573] _dn [-0.04980601]
up [0.00370084] _dn [-0.04946247]
up [0.00311631] _dn [-0.050047]
up [0.00288011] _dn [-0.0502832]
up [0.00229591] _dn [-0.0508674]
up [0.00287159] _dn [-0.05029172]
up [0.00333132] _dn [-0.049832]
up [0.00147048] _dn [-0.05169284]
up [0.00239418] _dn [-0.05076913]
up [0.00366751] _dn [-0.0494958]
up [0.00145624] _dn [-0.05170707]
up [0.00238092] _dn [-0.0507824]
up [0.00191089] _dn [-0.05125242]
up [0.00272109] _dn [-0.05044222]
up [0.0019014] _dn [-0.05126191]
up [0.00294553] _dn [-0.05021779]
up [0.00352363] _dn [-0.04963968]
up [0.00282048] _dn [-0.05034283]
up

up [0.00196598] _dn [-0.05154287]
up [0.00172947] _dn [-0.05177938]
up [0.00207357] _dn [-0.05143528]
up [0.00137255] _dn [-0.0521363]
up [0.00322574] _dn [-0.05028312]
up 0 _dn [-0.05353795]
up [0.00115588] _dn [-0.05238207]
up [0.0023142] _dn [-0.05122375]
up [0.0014961] _dn [-0.05204185]
up [0.00207315] _dn [-0.0514648]
up [0.00032394] _dn [-0.05321401]
up [0.00148275] _dn [-0.0520552]
up [0.00217719] _dn [-0.05136076]
up [0.00112477] _dn [-0.05241318]
up [0.0024017] _dn [-0.05113625]
up [0.00158216] _dn [-0.05195579]
up [0.00216016] _dn [-0.05137779]
up [0.00180655] _dn [-0.0517314]
up [0.00168586] _dn [-0.05185209]
up [0.00074951] _dn [-0.05278844]
up [0.00167721] _dn [-0.05186074]
up [0.00318747] _dn [-0.05035048]
up [0.00166884] _dn [-0.05186911]
up [0.00422301] _dn [-0.04931494]
up [0.00258943] _dn [-0.05094852]
up [0.00142428] _dn [-0.05211367]
up [0.00223249] _dn [-0.05130546]
up [0.0014158] _dn [-0.05212215]
up [0.00245593] _dn [-0.05108202]
up [0.00187144] _dn [-0.05166651]

up [0.00243805] _dn [-0.05355382]
up [0.00219881] _dn [-0.05379306]
up [0.00325083] _dn [-0.05274104]
up [0.00230749] _dn [-0.05368438]
up [0.00242038] _dn [-0.05357149]
up [0.00370797] _dn [-0.0522839]
up [0.00323378] _dn [-0.05275809]
up [0.00311209] _dn [-0.05287978]
up [0.00345959] _dn [-0.05253228]
up [0.00462609] _dn [-0.05136578]
up [0.0022793] _dn [-0.05371257]
up [0.0019231] _dn [-0.05406877]
up [0.00133115] _dn [-0.05466072]
up [0.0027345] _dn [-0.05325737]
up [0.00273024] _dn [-0.05326164]
up [0.00389968] _dn [-0.05209219]
up [0.00272184] _dn [-0.05327003]
up [0.00201335] _dn [-0.05397852]
up [0.00271322] _dn [-0.05327865]
up [0.00329598] _dn [-0.05269589]
up [0.00340882] _dn [-0.05258305]
up [0.00258325] _dn [-0.05340862]
up [0.00246158] _dn [-0.05353029]
up [0.00233978] _dn [-0.05365209]
up [0.0019828] _dn [-0.05400907]
up [0.00291838] _dn [-0.05307349]
up [0.0022089] _dn [-0.05378297]
up [0.00267477] _dn [-0.0533171]
up [0.00114047] _dn [-0.0548514]
up [0.00125147] _dn [-

up [0.00195323] _dn [-0.05492828]
up [0.00230102] _dn [-0.0545805]
up [0.00288364] _dn [-0.05399788]
up [0.00334821] _dn [-0.05353331]
up [0.0031088] _dn [-0.05377271]
up [0.00251854] _dn [-0.05436297]
up [0.00239714] _dn [-0.05448437]
up [0.00251002] _dn [-0.0543715]
up [0.00180284] _dn [-0.05507867]
up [0.00250156] _dn [-0.05437995]
up [0.00284873] _dn [-0.05403279]
up [0.00190726] _dn [-0.05497425]
up [0.00213723] _dn [-0.05474428]
up [0.00482566] _dn [-0.05205586]
up 0 _dn [-0.05744564]
up [0.00573328] _dn [-0.05171236]
up [0.00373691] _dn [-0.05370873]
up [0.00361524] _dn [-0.0538304]
up [0.00349381] _dn [-0.05395183]
up [0.00220559] _dn [-0.05524004]
up [0.00173334] _dn [-0.0557123]
up [0.00242971] _dn [-0.05501592]
up [0.00219118] _dn [-0.05525445]
up [0.00277188] _dn [-0.05467376]
up [0.00229923] _dn [-0.05514641]
up [0.00346586] _dn [-0.05397977]
up [0.0027591] _dn [-0.05468654]
up [0.00111434] _dn [-0.0563313]
up [0.00310217] _dn [-0.05434346]
up [0.00157285] _dn [-0.05587279

In [15]:
check.describe()

,close
count,5.000000
mean,-0.056582
std,0.025241
min,-0.091703
25%,-0.070099
50%,-0.049174
75%,-0.046631
max,-0.025303


In [ ]:
ffd_series.head()

In [ ]:
di = ffd_series.diff()
di

In [ ]:
vol

In [26]:
ffd_series[ffd_series['close'] < (-0.029416)]

,close
date_time,
2015-06-28 22:00:00.390,-0.035955
2015-07-05 22:06:35.981,-0.034766
2015-07-08 01:34:50.924,-0.029898
2015-08-07 13:26:50.986,-0.029644
2015-08-11 02:47:23.167,-0.032091
...,...
2016-12-30 20:59:58.848,-0.101709
2016-12-30 21:00:00.352,-0.102272
2016-12-30 21:00:24.294,-0.101718
